In [19]:
import os
import pandas as pd
pd.set_option('display.max_columns',None)
currentDirname = './code'
config = {
    'train_path':f'{currentDirname}/../data/used_car_train_20200313.csv',
    'test_path':f'{currentDirname}/../data/used_car_testB_20200421.csv',
    'prediction_path':f'{currentDirname}/../prediction_result/predictions.csv',
    'tmp_path':f'{currentDirname}/../user_data',
}

# 获取数据
train_data = pd.read_csv(config['train_path'],sep=' ')
test_data  = pd.read_csv(config['test_path'],sep=' ')

In [ ]:
import pandas as pd
from datetime import datetime
import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')
class generation:
    '''
    获取原始数据并进行特征工程，返回处理后的数据
    用法：
    gen = generation()
    change_data = gen.fit_transform(data)
    '''
    def fit_transform(self,data:pd.DataFrame):
        # -------------预处理----------------
        data['name_count'] = data.groupby(['name'])['SaleID'].transform('count')
        data.drop(labels=['name','price'],axis=1, inplace=True) # 去除名字这列
        fillnaList = ['bodyType', 'fuelType', 'gearbox','model']
        for x in fillnaList:
            data[x].fillna(0,inplace=True)
        data.power = data.power.apply(lambda x:x if x<600 else 600)
        data.notRepairedDamage = data.notRepairedDamage.astype('str').apply(lambda x:x if x!='-' else None).astype('float32')
        # ------------特征工程---------------
        def date_process(x:int):
            x = str(x)
            return datetime(int(x[:4]),max(1,int(x[4:6])),int(x[6:]))
        data.regDate = data.regDate.apply(date_process)
        data.creatDate = data.creatDate.apply(date_process)
        data['regDate_year'] = data['regDate'].dt.year
        data['regDate_month'] = data['regDate'].dt.month
        data['regDate_day'] = data['regDate'].dt.day
        data['creatDate_year'] = data['creatDate'].dt.year
        data['creatDate_month'] = data['creatDate'].dt.month
        data['creatDate_day'] = data['creatDate'].dt.day

        data['car_age_day'] = (data['creatDate'] - data['regDate']).dt.days
        data['car_age_year'] = round(data['car_age_day'] / 365, 1)

        # 地区
        data['regionCode_count'] = data.groupby(['regionCode'])['SaleID'].transform('count')
        data['city'] = data['regionCode'].apply(lambda x:str(x)[:2])

        # 分类特征（kilometer挺好的不用分）
        bin = [i*10 for i in range(-1,60)]
        data['power_bin'] = pd.cut(data['power'],bin,labels=False)

        bin = [i*10 for i in range(-1, 25)]
        data['model_bin'] = pd.cut(data['model'],bin,labels=False)

        # 匿名特征交叉融合
        for i in range(15): 
            for j in range(15):
                data['new_'+str(i)+'*'+str(j)] = data['v_' + str(i)]*data['v_' + str(j)]
                data['new_'+str(i)+'+'+str(j)] = data['v_' + str(i)]+data['v_' + str(j)]
                data['new_'+str(i)+'-'+str(j)] = data['v_' + str(i)]-data['v_' + str(j)]
        
        for i in range(15):
            data['new'+str(i)+'*power'] = data['v_'+str(i)]*data['power']
            data['new'+str(i)+'*day'] = data['v_'+str(i)]*data['car_age_day']
            data['new'+str(i)+'*year'] = data['v_'+str(i)]*data['car_age_year']

        # 特征筛选
        # ...
        
        return data

In [21]:
data = generation().fit_transform(pd.concat([train_data,test_data],axis=0))

In [22]:
used_features = data.columns.tolist()
used_features = [x for x in used_features if x not in ['SaleID','regDate','creatDate'] ]
# data = data[used_features]
print(data[used_features].describe())
data[used_features] = MinMaxScaler().fit_transform(data[used_features].values)

               model          brand       bodyType       fuelType  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean       47.258900       8.061335       1.738840       0.354190   
std        49.629518       7.873638       1.761835       0.539871   
min         0.000000       0.000000       0.000000       0.000000   
25%        11.000000       1.000000       0.000000       0.000000   
50%        30.000000       6.000000       1.000000       0.000000   
75%        66.000000      13.000000       3.000000       1.000000   
max       247.000000      39.000000       7.000000       6.000000   

             gearbox         power      kilometer  notRepairedDamage  \
count  200000.000000  200000.00000  200000.000000      167607.000000   
mean        0.216485     116.77520      12.597435           0.113492   
std         0.411850      70.17691       3.917803           0.317074   
min         0.000000       0.00000       0.500000           0.000000   
25%         0.0000

KeyboardInterrupt: 

In [ ]:
data.head()

,SaleID,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,name_count,regDate_year,regDate_month,regDate_day,creatDate_year,creatDate_month,creatDate_day,car_age_day,car_age_year,regionCode_count,city,power_bin,model_bin,new_0*0,new_0+0,new_0-0,new_0*1,new_0+1,new_0-1,new_0*2,new_0+2,new_0-2,new_0*3,new_0+3,new_0-3,new_0*4,new_0+4,new_0-4,new_0*5,new_0+5,new_0-5,new_0*6,new_0+6,new_0-6,new_0*7,new_0+7,new_0-7,new_0*8,new_0+8,new_0-8,new_0*9,new_0+9,new_0-9,new_0*10,new_0+10,new_0-10,new_0*11,new_0+11,new_0-11,new_0*12,new_0+12,new_0-12,new_0*13,new_0+13,new_0-13,new_0*14,new_0+14,new_0-14,new_1*0,new_1+0,new_1-0,new_1*1,new_1+1,new_1-1,new_1*2,new_1+2,new_1-2,new_1*3,new_1+3,new_1-3,new_1*4,new_1+4,new_1-4,new_1*5,new_1+5,new_1-5,new_1*6,new_1+6,new_1-6,new_1*7,new_1+7,new_1-7,new_1*8,new_1+8,new_1-8,new_1*9,new_1+9,new_1-9,new_1*10,new_1+10,new_1-10,new_1*11,new_1+11,new_1-11,new_1*12,new_1+12,new_1-12,new_1*13,new_1+13,new_1-13,new_1*14,new_1+14,new_1-14,new_2*0,new_2+0,new_2-0,new_2*1,new_2+1,new_2-1,new_2*2,new_2+2,new_2-2,new_2*3,new_2+3,new_2-3,new_2*4,new_2+4,new_2-4,new_2*5,new_2+5,new_2-5,new_2*6,new_2+6,new_2-6,new_2*7,new_2+7,new_2-7,new_2*8,new_2+8,new_2-8,new_2*9,new_2+9,new_2-9,new_2*10,new_2+10,new_2-10,new_2*11,new_2+11,new_2-11,new_2*12,new_2+12,new_2-12,new_2*13,new_2+13,new_2-13,new_2*14,new_2+14,new_2-14,new_3*0,new_3+0,new_3-0,new_3*1,new_3+1,new_3-1,new_3*2,new_3+2,new_3-2,new_3*3,new_3+3,new_3-3,new_3*4,new_3+4,new_3-4,new_3*5,new_3+5,new_3-5,new_3*6,new_3+6,new_3-6,new_3*7,new_3+7,new_3-7,new_3*8,new_3+8,new_3-8,new_3*9,new_3+9,new_3-9,new_3*10,new_3+10,new_3-10,new_3*11,new_3+11,new_3-11,new_3*12,new_3+12,new_3-12,new_3*13,new_3+13,new_3-13,new_3*14,new_3+14,new_3-14,new_4*0,new_4+0,new_4-0,new_4*1,new_4+1,new_4-1,new_4*2,new_4+2,new_4-2,new_4*3,new_4+3,new_4-3,new_4*4,new_4+4,new_4-4,new_4*5,new_4+5,new_4-5,new_4*6,new_4+6,new_4-6,new_4*7,new_4+7,new_4-7,new_4*8,new_4+8,new_4-8,new_4*9,new_4+9,new_4-9,new_4*10,new_4+10,new_4-10,new_4*11,new_4+11,new_4-11,new_4*12,new_4+12,new_4-12,new_4*13,new_4+13,new_4-13,new_4*14,new_4+14,new_4-14,new_5*0,new_5+0,new_5-0,new_5*1,new_5+1,new_5-1,new_5*2,new_5+2,new_5-2,new_5*3,new_5+3,new_5-3,new_5*4,new_5+4,new_5-4,new_5*5,new_5+5,new_5-5,new_5*6,new_5+6,new_5-6,new_5*7,new_5+7,new_5-7,new_5*8,new_5+8,new_5-8,new_5*9,new_5+9,new_5-9,new_5*10,new_5+10,new_5-10,new_5*11,new_5+11,new_5-11,new_5*12,new_5+12,new_5-12,new_5*13,new_5+13,new_5-13,new_5*14,new_5+14,new_5-14,new_6*0,new_6+0,new_6-0,new_6*1,new_6+1,new_6-1,new_6*2,new_6+2,new_6-2,new_6*3,new_6+3,new_6-3,new_6*4,new_6+4,new_6-4,new_6*5,new_6+5,new_6-5,new_6*6,new_6+6,new_6-6,new_6*7,new_6+7,new_6-7,new_6*8,new_6+8,new_6-8,new_6*9,new_6+9,new_6-9,new_6*10,new_6+10,new_6-10,new_6*11,new_6+11,new_6-11,new_6*12,new_6+12,new_6-12,new_6*13,new_6+13,new_6-13,new_6*14,new_6+14,new_6-14,new_7*0,new_7+0,new_7-0,new_7*1,new_7+1,new_7-1,new_7*2,new_7+2,new_7-2,new_7*3,new_7+3,new_7-3,new_7*4,new_7+4,new_7-4,new_7*5,new_7+5,new_7-5,new_7*6,new_7+6,new_7-6,new_7*7,new_7+7,new_7-7,new_7*8,new_7+8,new_7-8,new_7*9,new_7+9,new_7-9,new_7*10,new_7+10,new_7-10,new_7*11,new_7+11,new_7-11,new_7*12,new_7+12,new_7-12,new_7*13,new_7+13,new_7-13,new_7*14,new_7+14,new_7-14,new_8*0,new_8+0,new_8-0,new_8*1,new_8+1,new_8-1,new_8*2,new_8+2,new_8-2,new_8*3,new_8+3,new_8-3,new_8*4,new_8+4,new_8-4,new_8*5,new_8+5,new_8-5,new_8*6,new_8+6,new_8-6,new_8*7,new_8+7,new_8-7,new_8*8,new_8+8,new_8-8,new_8*9,new_8+9,new_8-9,new_8*10,new_8+10,new_8-10,new_8*11,new_8+11,new_8-11,new_8*12,new_8+12,new_8-12,new_8*13,new_8+13,new_8-13,new_8*14,new_8+14,new_8-14,new_9*0,new_9+0,new_9-0,new_9*1,new_9+1,new_9-1,new_9*2,new_9+2,new_9-2,new_9*3,new_9+3,new_9-3,new_9*4,new_9+4,new_9-4,new_9*5,new_9+5,new_9-5,new_9*6,new_9+6,new_9-6,new_9*7,new_9+7,new_9-7,new_9*8,new_9+8,new_9-8,new_9*9,new_9+9,new_9-9,new_9*10,new_9+10,new_9-10,new_9

In [ ]:
print(data.columns,data.columns.__len__())
for col in data.columns:
    print(f'{col} ({data[col].dtype})：缺失值总数={data[col].isna().sum()}')

Index(['SaleID', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'notRepairedDamage',
       ...
       'new11*year', 'new12*power', 'new12*day', 'new12*year', 'new13*power',
       'new13*day', 'new13*year', 'new14*power', 'new14*day', 'new14*year'],
      dtype='object', length=762) 762
SaleID (int64)：缺失值总数=0
regDate (datetime64[ns])：缺失值总数=0
model (float64)：缺失值总数=0
brand (float64)：缺失值总数=0
bodyType (float64)：缺失值总数=0
fuelType (float64)：缺失值总数=0
gearbox (float64)：缺失值总数=0
power (float64)：缺失值总数=0
kilometer (float64)：缺失值总数=0
notRepairedDamage (float64)：缺失值总数=32393
regionCode (float64)：缺失值总数=0
seller (float64)：缺失值总数=0
offerType (float64)：缺失值总数=0
creatDate (datetime64[ns])：缺失值总数=0
v_0 (float64)：缺失值总数=0
v_1 (float64)：缺失值总数=0
v_2 (float64)：缺失值总数=0
v_3 (float64)：缺失值总数=0
v_4 (float64)：缺失值总数=0
v_5 (float64)：缺失值总数=0
v_6 (float64)：缺失值总数=0
v_7 (float64)：缺失值总数=0
v_8 (float64)：缺失值总数=0
v_9 (float64)：缺失值总数=0
v_10 (float64)：缺失值总数=0
v_11 (float64)：缺失值总数=0
v_12 (

In [ ]:
print(data.describe())

              SaleID                        regDate          model  \
count  200000.000000                         200000  200000.000000   
mean   112499.500000  2003-10-13 21:47:02.688000128       0.191332   
min         0.000000            1991-01-01 00:00:00       0.000000   
25%     49999.750000            1999-10-01 00:00:00       0.044534   
50%     99999.500000            2003-09-12 00:00:00       0.121457   
75%    162499.250000            2007-11-07 00:00:00       0.267206   
max    249999.000000            2015-12-12 00:00:00       1.000000   
std     75346.610533                            NaN       0.200929   

               brand       bodyType       fuelType        gearbox  \
count  200000.000000  200000.000000  200000.000000  200000.000000   
mean        0.206701       0.248406       0.059032       0.216485   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.025641       0.000000       0.000000       0.000000   
50%         0.153846    

In [ ]:
# tmpData = data.copy()
data.columns.tolist()[:20]

['SaleID',
 'regDate',
 'model',
 'brand',
 'bodyType',
 'fuelType',
 'gearbox',
 'power',
 'kilometer',
 'notRepairedDamage',
 'regionCode',
 'seller',
 'offerType',
 'creatDate',
 'v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5']

模型

In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

Train_data = data[]